# Objectives

Anything that you do in the sea services depends fundamentally on the properties of fluids. If you fly aircraft, you maintain altitude because of the lift on the wings or blades of your craft. The depth your submarine cruises at is a result of how much air versus water the boat has in its ballast tanks. Whether you are a sailor or a marine, your ship stays on the water because a naval architect has ensured the ship is buoyant and maintains its stability in choppy weather. Air and water are two prominent examples of fluids, but there are many others you can probably think of.

It is important to realize that "buoyant force" refers to the total force a fluid exerts on an object; however, this force results from the difference in pressure on the top and bottom of the object. The pressure on the bottom is almost always greater, giving an upwards buoyant force. Therefore, this lesson starts with the idea of pressure, and builds on that to develop the buoyant force. It was the great thinker Archimedes who came up with a relatively simple way to find the buoyant force, over two thousand years ago. If you use the resulting Archimedes' principle, you will have an able guide to understand situations with fluids.

Here are the objectives for this lesson:

* Define pressure and density.
* State the density of fresh water.
* State the value of standard atmospheric pressure in SI units.
* Explain Pascal's principle.
* Explain the relationship between pressure and depth in a fluid.
* Solve for the column height of a fluid in a U-tube using Pascal's principle.
* State Archimedes' Principle.
* Calculate the buoyant force on submerged or floating objects.

# Fluids

## Definition of a fluid

A **fluid** is a substance that can readily flow from place to place and take on the shape of a container. Fluid can be a liquid or a gas. For any type of object, fluid or otherwise, we can define the **density** of the object. It has a symbol $\rho$ ("rho") and is defined by mass over volume, or

$$
    \rho = \frac{M}{V}
$$

The density has SI units of kg/m$^3$. *You should memorize the density of fresh water as 1000 kg/m$^3$.*

> **Problem:** A cube of metal has a mass of 347 g and measures 3.21 cm on a side. Using the table of densities given, identify the metal.
> 
> | Substance | $\rho$ (kg/m$^3$) |
> |-----------|-------------------|
> | aluminum  | 2700              |
> | iron      | 7860              |
> | silver    | 10500             |
> | lead      | 11300             |
> | mercury   | 13600             |
> | gold      | 19300             |

Historically, one of the more important ideas in physics from the 18th and 19th centuries is the [kinetic theory of gases](https://en.wikipedia.org/wiki/Kinetic_theory_of_gases), which explains many of the properties of a gas by the simple model of assuming the gas is made of very small particles moving at random speeds. We will explore this idea a little as we motivate how the pressure of a fluid can change based on the depth the pressure is measured at. A good simulation to explore this model, and to review how it accounts for the various gas laws, is given in the next cell.

In [ ]:
from IPython.display import IFrame

IFrame(src="https://phet.colorado.edu/sims/html/gases-intro/latest/gases-intro_en.html", width="600", height="400")

## A vPython model of a fluid

We will now construct our own simple vPython simulation of a gas. First, we need to import all the necessary vPython modules.

In [ ]:
from vpython import canvas, cos, curve, cylinder, dot, mag, norm, random, pi, rate, sin, sqrt, vector

To keep things visually simple, we will use a two-dimensional gas, so you can see everything that is happening at all times. This means we will have disks representing the molecules of the gas inside of a square box. There are several things we need to do to set up this simulation:

* Create the disks inside the box with an initial velocity
* Check for collisions between each pair of disks
* Check for collisions between the disks and the walls

In order to not lose the forest because of the trees, I will create each of these tasks as a separate function in vPython. The first creates the disks -- the basic idea is to create the disks at random locations. The function creates and returns a list `disk_object_list` of cylinders, each with a given position and velocity. To make sure disks do not form on top of each other, the ratio `AREA_RATIO` of the area of the disks to the total area is kept at a (fixed) small value. This is a relatively simple way of setting up the simulation, but more sophisticated methods are possible. Run the cell below, so the function can be used later.

In [ ]:
# Create disks

def createDisks():
    
    # Create disk radius as global variable, so that all other
    # functions can know about it
    
    global R
    
    # Set disk size. We want to make it easier to find a random list
    # of positions for the disks, so we must make sure that the area
    # of all the disks is less than the total area of the square. To
    # do this, we assume
    #
    #        (NUM_DISKS * pi * R^2) / (2 * L)^2 << 1
    #
    # Specficially, the ratio is set to the value AREA_RATIO, and the
    # resulting radius R is solved for.

    AREA_RATIO = 0.1
    R = L * sqrt(AREA_RATIO / (NUM_DISKS * pi))
    
    # Initial speed of all disks
    
    INIT_SPEED = 5
                    
    # Create list of disk objects
    
    disk_object_list = []
        
    for iii in range(NUM_DISKS):

        # Random position; the center of the disk must be
        # at least one disk radius away from box edge
        
        x = (L - R) * (1 - 2 * random())
        y = (L - R) * (1 - 2 * random())
        z = 0
        
        # Random direction for velocity; all disk speeds
        # are set to be the same value INIT_SPEED
        
        Q = 2 * pi * random()
        vx = INIT_SPEED * cos(Q)
        vy = INIT_SPEED * sin(Q)
        vz = 0
        
        # Put these into disk object list
        
        disk_object_list = disk_object_list + [cylinder(pos = vector(x, y, z), \
                            velocity = vector(vx, vy, vz), axis = vector(0, 0, 0.01), radius = R)]

    # Send list back to main procedure
    
    return disk_object_list

Next is a function to check whether the disks have hit the walls. This is relatively simple, and we have used something similar in previous simulations. Since the sides of the box run from `-L` to `L` in both the $x$ and $y$ directions, we just have to check whether the disk center `disk_pos` has gone *beyond* one disk radius `R` from the walls. This is the same as making sure that

$$
    |x| \ge (L - R) \qquad
    |y| \ge (L - R)
$$

Verify this for each wall if you are unsure of this!

In [ ]:
# Check for collisions against wall

def checkWallCollision(disk):
    
    # Check to see if the disk is too close to the walls;
    # flip the direction of the appropriate velocity
    # component if this does happen
    
    if abs(disk.pos.x) >= (L - R):
        disk.velocity.x = -disk.velocity.x
    if abs(disk.pos.y) >= (L - R):
        disk.velocity.y = -disk.velocity.y 
        
    # If a disk has gotten loose, use modular arithmetic
    # to put it back in the box on the inside
        
    if abs(disk.pos.x) > L:
        disk.pos.x = (disk.pos.x + 2 * L) % L
        
    if abs(disk.pos.y) > L:
        disk.pos.y = (disk.pos.y + 2 * L) % L
        
    # Return disk object with possibly changed
    # position, velocity vectors
        
    return disk

Finally, our third function checks for collisions between a pair of disks. Remember when we considered collisions in Lessons 10 and 12, that we use conservation of momentum to find the final velocities of the two disks after the collision. This is done in three steps:

* From the disk velocities that we see, use relative motion find the velocities of the disks in the center of mass (CM) frame.
* Use conservation of momentum in the CM frame. The collision is assumed to be elastic, since the disks are "hard", so the coefficient of restitution is $\epsilon = 1$. This means that the CM frame velocities for disks 1 and 2 are

$$
    {\vec v}_{1f, CM} = - {\vec v}_{1i, CM} \qquad
    {\vec v}_{2f, CM} = - {\vec v}_{2i, CM}
$$

* Take the final CM velocities and use relative motion again to find the final velocities that we see. Return these velocities as `disk1_vel` and `disk2_vel`.

In [ ]:
# Check for collisions between pairs of disks

def checkDiskCollision(disk1, disk2):
    
    # Collision occurs if the difference between the positions
    # is less than twice the individual disk radius
    
    if mag(disk1.pos - disk2.pos) <= 2 * R:
        
        # Transfer from lab frame to CM frame
        
        CM_vel = 0.5 * (disk1.velocity + disk2.velocity)
        disk1_CM_vel = disk1.velocity - CM_vel
        disk2_CM_vel = disk2.velocity - CM_vel
        
        # Using initial velocities and Newton's 2nd law in
        # momentum terms, find final velocities of disks
        
        distUnitVec = norm(disk1.pos - disk2.pos)
        disk1_CM_vel = disk1_CM_vel - 2 * dot(disk1_CM_vel, distUnitVec) * distUnitVec
        disk2_CM_vel = disk2_CM_vel - 2 * dot(disk2_CM_vel, distUnitVec) * distUnitVec
        
        # Transfer from CM frame back to lab frame
        
        disk1.velocity = disk1_CM_vel + CM_vel
        disk2.velocity = disk2_CM_vel + CM_vel
        
    return [disk1, disk2]

Now, we can put this all together into our main procedure, which is given in the cell below. Run the cell, and you should see our gas of disks merrily moving around a box, colliding with each other and the walls.

In [ ]:
canvas()

# Constants

L = 2
NUM_DISKS = 20
MAX_TIME = 5
t = 0
dt = 0.01

# Create disks with random starting positions, velocities

diskList = createDisks()

# Create boundary of square for disks

boundary = curve(radius = 0.01)
boundary.append(vector(L, L, 0), vector(L, -L, 0), vector(-L, -L, 0), vector(-L, L, 0), vector(L, L, 0))

# Time evolution

while t < MAX_TIME:
    rate(100)
    
    # Go through all the disks, check for collisions, and
    # then update the position, velocity of the disk
    
    for jjj in range(NUM_DISKS):
    
        # Check for wall collisions
    
        diskList[jjj] = checkWallCollision(diskList[jjj])
        
        # Check for disk collisions; to save computational
        # time, only check for collisions with disks having
        # numerical labels greater than the current disk
        
        for kkk in range(jjj + 1, NUM_DISKS):
            [diskList[jjj], diskList[kkk]] = checkDiskCollision(diskList[jjj], diskList[kkk])
        
        # Update positions, velocities in lists
        
        diskList[jjj].pos = diskList[jjj].pos + diskList[jjj].velocity * dt
        
    # Update time
    
    t = t + dt

Sometimes, if you run many simulations in the same Jupyter notebook, your computer may slow down. This is partly because the computer may keep track of "old" vPython objects -- such as the disks in our gas simulation -- even though they are no longer being displayed. The procedure below acts as a measure to remove previous disks from memory. If you run the second cell, you will see the disks in the `canvas` above disappear!

In [ ]:
# For any object 'thing', make
# it invisible and delete it

def cleanup(thing):
    try:
        thing.visible = False
        del thing
    except:
        pass

In [ ]:
# Run through the disk objects
# of the simulation and clean
# them up using above procedure
    
for disk in diskList:
    cleanup(disk)

The default number of disks `NUM_DISKS` in the simulation above is set to 20, so that the simulation runs smoothly. If you try to increase this number too much, the disks will move slowly. This is because, if the number of disks is $N$, then the number of times the computer must check for disk collisions is roughly $N^2$. For example, doubling the number of disks means the computer has to do about four times as many calculations. 

This is where you go from "programming" to "computer science", by developing smarter algorithms to handle the disk collision check. In this way, the vPython script run in the cell below uses various methods to try and speed up calculations for the disk gas. Thus, it can run roughly the same speed with a greater number of disks.

In [ ]:
%run ../scripts/2D_disk_gas.py

In [ ]:
# Clean up disks from script
    
for disk in diskObjList:
    cleanup(disk)

# Pressure

## Definition of pressure

Now that we have a simple model of a fluid, let's use it to get an intuition for how fluids generate pressure. Consider what happens when you push both a pin and the blunt end of a pen against your skin with the same force. What will determine whether your skin will be punctured? Or how you "feel" this push? It turns out your skin actually measures the amount of force over a given area, not the absolute size of the force. So, if you exert the same force, but over a larger portion of your body, it does not feel as painful as if you apply the force to a small area.

Thus, the ratio of force over area is important for determining how painful such prods are, among a lot of other things! The ratio is called the **pressure**.

> * **Symbol:** $P$
> * **Definition:**
>
> $$ P = \frac{F}{A} $$
>
> * **SI units:** Pascals
>
> $$ 1 \textrm{ Pa} = 1\ \frac{\textrm{N}}{\textrm{m}^2} $$

For example, squeezing an egg from all sides will *not* break the egg. The curved shape of the egg is able to redistribute the force all across its area, so the resulting pressure on the egg is too small to break the shell. However, exerting a force on a small area -- by hitting the egg against the edge of a bowl -- results in a large enough pressure to crack the egg!

> **Problem:** You are walking out on a frozen lake and you begin to hear the ice cracking beneath you. What is your best strategy for getting off the ice safely?
>
> 1. Jump up and down to lessen your contact time with the ice.
> 1. Lie down flat on the ice and crawl toward shore.
> 1. Shuffle your feet (without lifting them) to move toward shore.
> 1. Stand absolutely still and wait for rescue.

![A tire gauge is used to measure the difference in the pressure inside the tire *above* atmospheric pressure](../img/INT_AY20_MP2_L06_Fig01-Tire_gauge_small.png)

There are other non-SI units that you will see when dealing with pressure. One unit is the *atmosphere* (atm), which is the standard average pressure at sea level on the Earth's surface. We will deal more with atmospheric pressure below. Another such unit is *pounds per square inch* (psi). The conversion factors between these units are listed below.
    
* Atmospheres -- 1 atm = 101.5 kPa
* 1 atm = 14.7 $\frac{\text{lb}}{\text{in}^2}$ (psi)

Most tire gauges measure the pressure of vehicle tires in psi. You might also see units of *mm of Hg* (Torr). Torr are most often used when measuring blood pressure or by meteorologists discussing the weather!

![Two types of heel (high and flat) give different pressures on the foam beneath them](../img/INT_AY20_MP2_L06_Fig02-Shoes_on_foam_small.png)

> **Problem:** A 60.0 kg woman is standing up such that the weight is distributed equally on both feet. If her heels are supporting 75.0 percent of her weight and the contact area between her heels and the ground are 0.250 in$^2$ (for the high heel) and 2.00 in$^2$ (for the flat heel), what is the pressure value on each heel? *Answers:* 1.37 MPa (high heel), 171 kPa (flat heel)

## Atmospheric pressure

Although we do not notice it, the air around us exerts a pressure at all times. This is known as the **atmospheric pressure**, and is due to the weight of the air above. If you live at a high altitude, there is less air above you, so the atmospheric pressure decreases. 

* Notation: $P_{at}$ -- atmospheric pressure; the **standard** value is
$$
    P_{at} = 1\ \textrm{atm} = 101.3 \ \textrm{kPa} = 14.7\ \frac{\text{lb}}{\text{in}^2}
$$
* If a problem does not give the atmospheric pressure, you may assume a standard pressure of 101.3 kPa.

![A Goethe barometer, originally developed by the German writer Johann Wolfgang Von Goethe ([source](https://commons.wikimedia.org/wiki/File:Barometer_Goethe_01.jpg))](../img/Barometer.png)

You can measure the atmospheric pressure by using a device called a barometer. A barometer is formed when a tube filled with fluid and blocked at both ends is put into a basin of the same fluid, and the bottom end is uncapped. The fluid level in the top of the tube will go down to some level and stay. Why is this?  Based on Aristotle's ideas -- "nature abhors a vacuum" -- it was originally believed that a vacuum was formed and could pull the fluid up with a limited amount of force. However, this idea was tested by various people in the 1630s and 1640s. Evangelista Torricelli thought instead it was the weight of the air itself *pushing* the fluid up, until the forces from the air and fluid weights balanced.

![Portrait of Evangelista Torricelli ([source](https://commons.wikimedia.org/wiki/File:Evangelista_Torricelli_by_Lorenzo_Lippi_(circa_1647,_Galleria_Silvano_Lodi_%26_Due).jpg))](../img/INT_AY20_MP2_L06_Fig03-Evangelista_Torricelli_small.png)

This idea was tested by Blaise Pascal in various ways in 1644, and it was shown that Torricelli was correct. One way Pascal did this was to convince his brother-in-law to climb a local mountain with a barometer, and record the heights of the fluid in the barometer at various altitudes. His brother-in-law did this, accompanied by several of the local dignitaries, and they saw a clear decrease in the fluid height.

![Portrait of Blaise Pascal ([source](https://commons.wikimedia.org/wiki/File:Blaise_Pascal_Versailles.JPG))](../img/INT_AY20_MP2_L06_Fig04-Blaise_Pascal.png)

If we want to measure the pressure in our car or bicycle tires, we can use a pressure gauge. Note that the gauge does *not* measure the absolute pressure. It is much easier to measure the difference in pressure between that in the tire and the air outside. This is because this difference can be quickly measured by finding the net force on a given area. The force will be the difference in the forces from the atmospheric pressure, and from the pressure inside the tire. Thus, **gauge pressure** is the difference between total pressure and atmospheric pressure for a given situation,

$$ 
    P_g = P_{tot} - P_{at}
$$

where $P_g$ is gauge pressure, $P_{tot}$ is total pressure and $P_{at}$ is atmospheric pressure.

> **Problem:** A glass bottle of soda is sealed with a screw cap. The absolute pressure of the carbon dioxide inside the bottle is 160 kPa. Assuming the top and bottom surfaces of the cap each have an area of 4.74 cm$^2$, obtain the magnitude of the force (in N) the screw thread exerts on the cap in order to keep it on the bottle. The air pressure outside the bottle is one atmosphere. *Answer:* 28.0 N

![2019 Chevrolet Corvette ZR1 at the Montreal International Car Show ([source](https://commons.wikimedia.org/wiki/File:2019_Chevrolet_Corvette_ZR1_au_SIAM_2019.jpg))](../img/2019_Chevrolet_Corvette.png)

> **Problem:** The weight of your 1420 kg car (2019 Corvette ZR1) is supported equally by its four tires, each inflated to a gauge pressure of 35.0 psi.  What is the area of contact each tire makes with the road (in cm$^2$)? *Answer:* 144 cm$^2$

# Pressure at a depth

If you measure the pressure at a particular point in a fluid, you are measuring the weight of all the fluid above that point acting on the area of your pressure gauge. Thus, for a denser fluid, the weight would increase, even if nothing else changed, since there is more mass above the point. Similarly, if you take the container of fluid to another planet, with a different gravitational field, the weight will change for that reason. Finally, if you change the depth of the point you are measuring, there will be more or less fluid above that point, giving a different pressure reading. Notice that moving horizontally will *not* change the pressure, since the amount of fluid above the point has not changed.

![Pressure at a depth in a fluid](../img/INT_AY20_MP2_L07_Fig01-Pressure_at_a_depth_small.png)

We can work out this pressure $P$ at a given depth $h$ in the fluid by imagining we have an area $A$ that we are measuring the pressure with -- this may be the surface of our pressure gauge, for example. The weight of fluid above that area will be due to all the mass above it. To find this mass, we can think of it as forming a cylinder above the area $A$, with a height the same as the depth $h$. Thus, the volume of this cylinder is $Ah$. If we use the fluid density $\rho$, then the *mass* of fluid above the area $A$ is

$$
    m = \rho V = \rho A h
$$

This means that there is a gravitational force on the pressure gauge area $A$ given by $F_g = mg = \rho g A h$. However, if we find the actual pressure on our gauge area $A$, the area cancels out! This gives us that the change in pressure $\Delta P$ due to a depth $h$ of fluid with density $\rho$ is

$$
    \Delta P = \frac{F_g}{A} = \frac{\rho g A h}{A} = \rho g h
$$

So, for example, imagine you are swimming in a pool. At a particular depth, the total pressure on you will consist of the pressure due to the water, and then the atmospheric pressure on the surface of the pool's water. Adding these together gives the total pressure $P_{tot$},

$$
    P_{tot} = P_{at} + \rho g h
$$

In this formula, we have the following variables:

* $P_{tot}$ -- total pressure at depth $h$ below the surface 
* $P_{at}$ -- total pressure at the surface of the fluid (typically atmospheric pressure is used)
* $\rho$ -- density of fluid
* $g$ -- magnitude of acceleration due to gravity (9.81 m/s$^2$ on Earth's surface)
* $h$ -- depth below the surface of the fluid
                                                   
This equation works if you have a container of fluid *open to the atmosphere*, with only one fluid in it. You can easily imagine a situation where more than one fluid is in the same container. There are many fluids which do not mix, and can form separate layers on top of each other, such as oil and water (how would you know what the order of the layers is, from top to bottom?). In that case, at the bottom of the container, the total pressure would be the sum of all the changes in pressure from each of the fluids, along with the atmospheric pressure, if the container is open. Then,

$$
    P_{tot} = P_{at} + \sum_i \Delta P_i = P_{at} + \rho_1 g h_1 + \rho_2 g h_2 + \cdots
$$

where $\rho_i$ is the density of fluid $i$, and $h_i$ is the depth of the fluid layer (*or* how deep the point is within that fluid, for the fluid the measuring point is inside).

One thing you may have noticed is that there was no mention of the container's shape at all! If the fluid is at rest, then the forces of the container wall on the fluid have to balance the fluid's force (coming from the pressure) back on the wall, due to Newton's 3rd law. Thus, the shape of the container does not matter at all when finding the pressure in a fluid, only the depth the measuring point is below the top of the fluid. You can explore pressure at a depth using the PhET linked in the next cell.

In [ ]:
from IPython.display import IFrame

IFrame(src="https://phet.colorado.edu/sims/html/under-pressure/latest/under-pressure_en.html", width="600", height="400")

> **Problem:** The figure below shows four identical boxes held stationary in a pool of water.  Which choice below correctly ranks the fluid pressure on the bottom of each block, from greatest to least?
>
> ![](../img/INT_AY20_MP2_L07_Fig02-Rank_the_boxes_small.png)

> **Problem:** Water in the lake behind Hoover Dam is 221 m deep. What is the *water pressure* (aka gauge pressure) at the base of the dam (in MPa)? *Answer:* 2.17 MPa
>
> ![Hoover dam seen from the air ([source](https://commons.wikimedia.org/wiki/File:Hoover_dam_from_air.jpg))](../img/Hoover_dam.png)

> **Problem:** The Mariana Trench is located in the Pacific Ocean at a depth of about 11.7 km below the surface of the water. The density of sea water is 1025 kg/m$^3$. If an underwater vehicle were to explore such a depth, what force (in MN) would the water exert on the vehicle's observation window, with a radius of 13.0 cm? *Answer:* 6.25 MN
>
> ![The U.S. Navy bathyscaphe *Trieste*, just before her record dive to the bottom of the Marianas Trench, 23 January 1960. The waves were were about 1,5 to 1,8 m high when scientist Jacques Piccard and Lt. Don Walsh, USN, and Piccard boarded from the rubber raft attached to the bathyscaphe. The destroyer escort USS *Lewis* (DE-535) is steaming by in the background. ([source](https://commons.wikimedia.org/wiki/File:Hoover_dam_from_air.jpg))](../img/Bathyscaphe_Trieste.png)

# Pascal's principle

As mentioned above, for a stationary fluid, the shape of the container does not matter when measuring the pressure at a given depth. Let's explore this notion some more. We can think of the fluid as composed of imaginary bags, each of which is pressing against its neighbors, or the container walls. In equilibrium, these forces will balance out, and the fluid remains stationary. If they did *not* balance out, then the net force on each "bag of fluid" would give it an acceleration until the forces *did* balance out, and equilibrium is restored.

Now, suppose there is an outside force acting on the fluid. This could be something like a piston pressing down on the fluid, or even a change in atmospheric pressure above a lake as a storm front comes into the area. This results in a change in the force on the top of the fluid. The "bags of fluid" will then rearrange themselves to account for this different force, until the fluid is again in equilibrium. The upshot is that an external pressure applied to an enclosed fluid is transmitted unchanged to every point in the fluid. This is known as **Pascal's principle**. One consequence is that the ressure of a stationary enclosed fluid at one vertical height is the same pressure at a different point with the same vertical height. If it were not, the fluid would flow until the pressures are equalized, and the net force on each "bag of fluid" is now zero.

![Pascal's principle says the pressure throughout the pool changes due to an external change in pressure](../img/INT_AY20_MP2_L06_Fig05-Diver_in_pool.png)

Let's go through an example, to see how this principle has practical consequences. Suppose we have a U-shaped tube, as shown in the diagram below, filled with water at the bottom, and oil on one side of the tube.

![U-tube filled with water](../img/INT_AY20_MP2_L07_Fig03-U_tube_small.png)

We are interested in finding the height $h$ of the oil column, if we know that the oil has a density of 800. kg/m$^3$. We use Pascal's principle to figure out this distance. Imagine we have two pressure gauges. If we place them side by side at the bottom of the tube, they should read the exact same pressure. Now let's raise the gauges upward, one of either side of the tube, so that they are always in the same depth of water inside the tube. Because they are in the same fluid at the same depth, they will still read the same pressure. This is true *until* the gauge going up the right side crosses into the oil; because the oil has a different density, the pressures will now be different on either side of the tube. So, at the highest point where there is still water on both sides of the tube, we can say that the pressures are equal:

$$
    P_{left} = P_{right}
$$

Now, we need to write out what the pressures are on either side at this particular depth. On the right-hand side, the pressure is due to the atmosphere, and the depth $h$ of oil above the water; on the left side, it is the atmosphere along with the depth $(h - d)$ cm of water, where $d = 2.00$ cm is the difference in surface height. This means that saying the pressures are equal on either side of the tube means

$$
    P_{at} + \rho_w g (h - d) = P_{at} + \rho_o g h
$$

where $\rho_w, \rho_o$ are the densities of water and oil, respectively. Cancelling out the two atmospheric pressures *first*, then cancelling out the factors of $g$, we can solve for the depth $h$ as

$$
    h = \biggl( \frac{\rho_w}{\rho_w - \rho_o} \biggr) d
$$

Notice what happens: the depth $h$ is determined by a ratio of densities, multiplied by the difference in surface depths $d$. Thus, as long as the units for the densities are the same, the units for $d$ and $h$ will also be the same -- there is no need to convert! Plugging in the numbers gives $h = 10.0$ cm.

> **Problem:** Technically, the two atmospheric pressures are *not* the same on either side, because of the slight difference in the surface height in the tube. Take this difference into account, using the density of air $\rho_a = 1.29$ kg/m$^3$ (at $0^\circ$ C), and see how much of a change there is in your answer for $h$. Calculate your answer to four sig figs. *Answer:* 9.987 cm (or a 0.13% difference)

> **Problem:** A U-shaped tube is filled mostly with fresh water, but a small amount of vegetable oil has been added to both sides. Calculate the density of the oil in kg/m$^3$, using the distances $h_1 = 4.00$ cm, $h_2 = 9.00$ cm, and $h_3 = 3.00$ cm. *Answer:* 400. kg/m$^3$
>
>
> ![U-tube filled with water and vegetable oil](../img/INT_AY20_MP2_L07_Fig04-U_tube_small.png)

As a military application of Pascal's principle, we look at the action of torpedoes. Torpedoes do not sink ships by holing their hulls, but instead are fired underneath the keel of the target. When the torpedo detonates, it creates a pressure wave, which forms the boundary of a steam bubble, lifting the hull and weakening the keel. The steam bubble quickly collapses, dropping the ship downward and breaking the keel. For more information, see the chapter [Underwater Weapons](https://fas.org/man/dod-101/navy/docs/es310/uw_wpns/uw_wpns.htm) at the on-line [Introduction to Naval Weapons](http://fas.org/man/dod-101/navy/docs/es310/syllabus.htm).

![How a torpedo breaks a ship's keel](../img/INT_AY20_MP2_L06_Fig06-Torpedo_effect_on_ship.png)

In June 1999, the Australian submarine HMAS Farncomb conducted a live-fire sinking exercise against the former HMAS Torrens, a destroyer escort. The video linked in the cell below clearly shows the intended effects of a torpedo on a target ship.

In [ ]:
from IPython.display import VimeoVideo

VimeoVideo(130514745, width=600, height=400, muted=1)

# Buoyancy

## Archimedes' principle

We now have the pieces we need to find the buoyancy force on an object in a fluid. Suppose we put a cylinder into a container. The fluid will exert a pressure on all sides of this cylinder. In particular, the pressure is greater on the bottom of the object than it is on the top, because the bottom is at a greater depth. This means there is a greater force $F_{bot}$ on the bottom surface area $A_{bot}$ than the force $F_{top}$ the fluid exerts on the top area $A_{top}$. The buoyant force will be upward because of this difference. Because we are using a cylinder for simplicity, $A_{bot} = A_{top} = A$; we can do the same thing for more complicated shapes, but it requires more math. In addition, the forces on the side will not matter, since the forces on opposite sides of the cylinder middle will cancel out.

![Forces acting on a completely submerged object](../img/INT_AY20_MP2_L08_Fig02-Immersed_block_with_forces_small.png)

Now, suppose the top of the cylinder is at a depth $H_{top}$. If the cylinder has a height $h$, then the bottom of the cylinder will be at a depth $H_{bot} = H_{top} + h$. Let's use this to find the net force on the cylindrical object, due to the fluid. Using the relations between force and pressure, and the method to find pressure at a depth, we have

$$
    \sum F_y = F_{bot} - F_{top} = (P_{bot} A) - (P_{top} A) = (P_{at} + \rho g H_{bot}) A - (P_{at} + \rho g H_{top}) A
$$

Cancelling out the common atmospheric pressures, we get

$$
    \sum F_y = \rho g (H_{bot} - H_{top}) A = \rho g h A
$$

However, the quantity $hA$ is just the volume of the cylinder! So the net force of the fluid on the object, which we call the **buoyant force** $F_B$, is given by

$$
    F_B = \rho_{fl} g V_{obj.\ in\ fluid}
$$

There are a couple of important things to note here:

* The density $\rho$ is the density of the fluid. Remember that this came in because we found the forces on the object due to being at a depth in the fluid.
* The volume $V$ is the volume of the object in the fluid. For the derivation above, we assumed that the object is completely submerged, but it also works for partially submerged objects, i.e. when the top is outside the fluid.

Because of these facts, the combination $\rho V$ is actually the mass of the fluid pushed aside by the object, so $\rho g V$ is the weight of the displaced fluid! This gives us a derivation of Archimedes' principle:

>**Archimedes' principle:** An object immersed (either completely or partially) in a fluid experiences an upward buoyant force equal in magnitude to the weight of fluid displaced by the object.

> **Problem:** Imagine holding two identical bricks in place under water, as shown in the diagram below. Brick 1 is just beneath the surface of the water, while brick 2 is held about two feet down. The force needed to hold brick 2 in place is <u>&emsp;&emsp;&emsp;</u> the force for brick 1.
>
> 1. greater than
> 1. less than
> 1. equal to
>
>![Two immersed blocks](../img/INT_AY20_MP2_L08_Fig03-Two_immersed_blocks_small.png)

Here is some advice when finding the buoyant force using Archimedes' principle. This idea is very powerful, but you also need to be very careful about the quantities involved. Typically, you know the density of the fluid, and maybe the density of the object as well. You may also know both the total volume of the object, and the amount of this volume that is in the fluid (e.g. for a floating object, these will be different). You need to make sure that you are matching the right quantities when finding the weight of the displaced fluid. This will depend on the fluid density, and the volume of fluid displaced by the object. This will give rise to a force on the object, which has its own density and total volume. Don't get these mixed up!

To see an example of this, let's imagine we have a spherical balloon with a diameter of 4.80 m. The skin of the balloon is made of neoprene synthetic rubber, and has a mass of 1.08 kg. The balloon is filled with 30.0$^\circ$ air, with a density of 1.16 kg/m$^3$, and the air outside the balloon is at room temperature (20.0$^\circ$) with a density of 1.20 kg/m$^3$. If you want to put an electronic instrument onto the balloon as cargo, what is the maximum mass it could have?

First, we need to sort everything out, by having different symbols for each of our quantities. Thus, we have

* $m_{skin}$ = 1.08 kg
* The volume of a sphere with radius $R$ is given by $(4/3) \pi R^3$; here, $R = 2.40$ m, so $V = 57.9$ m$^3$.
* $\rho_{in}$ = 1.16 kg/m$^3$
* $\rho_{out}$ = 1.20 kg/m$^3$

There will be several forces acting on the balloon: the upward buoyant force $F_B$ from the outside air, and the weights of the cargo $F_{g, cargo}$, the skin $F_{g, skin}$, *and* the inside air $F_{g, in}$ (all downward). Remember that the air inside the balloon also has a weight, and needs to be included! It is the small difference in density between the inside and outside air that allows a buoyant force, lifting the balloon itself along with its cargo. Thus, the FBD for the balloon looks like

![FBD for the balloon](../img/INT_AY21_L17_Fig01_FBD_small.png)

Starting from Newton's 2nd law

$$
    \sum F_y = F_B - F_{g, cargo} - F_{g, skin} - F_{g, in} = m_{tot} a_y
$$

we use the gravitational and buoyant force equations as appropriate:

$$
    \sum F_y = \rho_{out} g V - m_{cargo} g - m_{skin} g - \rho_{in} g V = m_{tot} a_y
$$

The balloon will rise into the air only if $a_y \ge 0$. Thus, we need to find the cargo mass $m_{cargo}$ that gives

$$
    \rho_{out} g V - m_{cargo} g - m_{skin} g - \rho_{in} g V \ge 0
$$

Dividing by $g$ does not change this inequality; solving for the cargo mass gives

$$
    m_{cargo} \le (\rho_{out} - \rho_{in}) V - m_{skin}
$$

The maximum cargo mass is *equal* to the right-hand side, i.e. when the inequality becomes an equality. Plugging in the numbers gives $m_{cargo, max}$ = 1.24 kg.

## Floating objects

Obviously, one important application for Archimedes' principle in the sea services is making sure that ships remain afloat (and submarines can maintain cruising depth). In this case, the net force on the ship must be zero, which means that the buoyant force is the same as the ship's weight.

![Forces acting on a floating object](../img/INT_AY20_MP2_L08_Fig04_Floating_block_small.png)

For this to happen, the floating object is in equilibrium, so forces balance:

$$
    F_b = F_{g, obj.}
$$

By Archimedes' principle, the buoyant force is equal to the weight of the displaced fluid:

$$
    F_b = F_{g, fluid}
$$

Putting these equations together gives for a *floating* object that

$$
    F_{g, obj} = F_{g, fluid}
$$

Thus, a floating ship must displace exactly its weight in water in order to stay on the surface. This is why, instead of giving the ship's mass, this statistic is listed as the ship's "displacement" (in tons, usually). For submarines, you may see two different displacements -- surface and submerged. The first, like that of the ship, gives the amount of water equivalent to the sub's *mass*; the second gives the amount of water with the same *volume* as the sub. Which would you expect to be bigger?

> **Problem:** Two identical blocks are floating in two separate aquariums. Aquarium A is filled with alcohol ($\rho$ = 789 kg/m$^3$), while aquarium B is filled with pure water. Which has the greater buoyant force acting on it?
>
> 1. The block floating in alcohol
> 1. The block floating in pure water
> 1. They both have the same buoyant force
> 1. Cannot determine without the density of the blocks

![The *Occitania* and a 2,500-ton barge carrying a load of coal destined for a thermal power plant ([source](https://commons.wikimedia.org/wiki/File:Barge_%C3%A0_charbon.jpg))](../img/Barge.png)

> **Problem:** You are shopping online for a new barge at [McDonough Marine Services](http://www.mcdonoughmarine.com/ocean-barges.html), and settle on a 140' $\times$ 40' $\times$ 9' 6'' size barge. Below is given the data for the *loadline* and *freeboard* loading for the barge:
>
> <table>
> <tr>
>     <th></th>
>     <th colspan = "4">Metric ton cargo at freeboards of</th>
> </th>
> <tr>
>     <th>Cargo at loadline (mt)</th>
>     <th>3'</th>
>     <th>4'</th>
>     <th>5'</th>
>     <th>6'</th>
> </tr>
> <tr>
>     <td>965</td>
>     <td>820</td>
>     <td>660</td>
>     <td>550</td>
>     <td>340</td>
> </tr>
> </table>
> 
> The **freeboard** is the distance from the deck of the barge down to the water line. The **loadline** is the freeboard when the maximum safe cargo is placed on the barge, so that it is the highest water line for safe operations. A **metric ton** (mt) is 1000 kg.
> 
> 1. What is the mass of the barge in metric tons? Use a density of 1010 kg/m$^3$ for water.
> 1. Where is the loadline located on the barge? Give the distance in cm from the barge deck.

![An iceberg in the Arctic Ocean ([source](https://commons.wikimedia.org/wiki/File:Iceberg_in_the_Arctic_with_its_underside_exposed.jpg))](../img/Iceberg.png)

> **Problem:** An iceberg has a density of 917 kg / m$^3$ and is floating in sea water ($\rho = 1025$ kg / m$^3$). What fraction $V_{sub} / V_{tot}$ of the iceberg is submerged in the sea water?

# Summary

Fluids are an important topic for those in the sea services. It can also be challenging to think through some of the puzzles you encounter with buoyancy. To emphasize the point: this is one of those topics where you really need to think through the consequences of the definitions and Archimedes' principle. Some of these consequences may be counterintuitive!

After this lesson, you should be able to:

* Define pressure, and identify its units (SI and otherwise).
* Use the definition of pressure to find an unknown force or area.
* Define atmospheric and gauge pressures.
* Find the pressure on an object at a certain depth inside of a fluid
* Use Pascal's principle and the relation for pressure at a given depth in a fluid to find the height or density of a fluid in a U-tube.
* Describe the relationship between the portion of an object's volume in a fluid, and the amount of fluid displaced by that object.
* State Archimedes' principle.
* Use Archimedes' principle to find the amount of an object that is immersed in a fluid.